## MXNet (with Gluon) を使った学習と推論を SageMaker で行う

#### ノートブックに含まれる内容

- MXNet および Gluon を SageMaker で使うときの，基本的なやりかた

#### ノートブックで使われている手法の詳細

- アルゴリズム: CNN
- データ: MNIST

## セットアップ

In [ ]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

## データのロード

Gluon に備わっている MNIST クラスを使って，データを取得し，これを S3 にアップロードします．SageMaker の学習時につかうデータは，S3 に置く必要があります．ここでは，ローカルにある iris データをいったん SageMaker SDK の session クラスにある upload_data() メソッドを使って，ノートブックインスタンスのローカルから S3 にアップロードします．

デフォルトでは SageMaker は sagemaker-{region}-{your aws account number} というバケットを使用します．当該バケットがない場合には，自動で新しく作成します．upload_data() メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することが可能です．

In [ ]:
gluon.data.vision.MNIST('./data/train', train=True)
gluon.data.vision.MNIST('./data/test', train=False)

また以下を実行する前に，**<span style="color: red;"> `data/mnist/XX` の `XX` を指定された適切な数字に変更</span>**してください

In [ ]:
inputs = sagemaker_session.upload_data(path='data', key_prefix='data/mnist/XX')

## Gluon を実行するスクリプトの中身を確認

MXNet のスクリプトは，学習時には以下の 2 つのメソッドを実装する必要があります．

- `train`: モデルを記述するメソッド
- `save`: 学習したモデルの保存処理を記述するメソッド

また，推論時には以下のメソッド群を実装する必要があります．

- `model_fn`: コンテナ起動時に，モデルをロードする処理を記述するメソッド
- `transform_fn`: invokeEndpoint が叩かれた際に呼ばれる，予測処理を記述するメソッド

さらに，MXNet の Module モデルや，Gluon を使用している場合には，上の `transform_fn` を，以下の 3 つのメソッド群に分けて記述することも可能です．
推論時に以下のメソッド群を記述することが可能です．これらはオプショナルなものなので，記述しなくても構いません．詳細は[こちら](https://github.com/aws/sagemaker-python-sdk#model-serving)をご覧ください．

- `input_fn`: 前処理を記述するメソッド
- `predict_fn`: 予測処理を記述するメソッド
- `output_fn`: 後処理を記述するメシッド

今回使用するスクリプトは，[Gluon MNIST example](https://github.com/apache/incubator-mxnet/blob/master/example/gluon/mnist.py) をベースにしています．以下で中身を実際に確認してみましょう．


In [ ]:
!cat 'mnist.py'

## モデルの学習を実行

SageMaker SDK には，MXNet 専用の Estimator として，`sagemaker.mxnet.MXNet` クラスがあります．ここでは，先ほどの `mnist.py` をエントリーポイントとして指定して，MXNet の学習ジョブを実行します．

In [ ]:
m = MXNet("mnist.py", 
          role=role, 
          train_instance_count=1, 
          train_instance_type="ml.m4.xlarge",
          hyperparameters={'batch_size': 100, 
                         'epochs': 20, 
                         'learning_rate': 0.1, 
                         'momentum': 0.9, 
                         'log_interval': 100})

In [ ]:
m.fit(inputs)

# モデルの推論を実行


推論を行うために，まず学習したモデルをデプロイします．`deploy()` メソッドでは，デプロイ先エンドポイントのインスタンス数，インスタンスタイプを指定します．モデルのデプロイには 10 分程度時間がかかります．

In [ ]:
predictor = m.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

それでは，以下の四角の中に実際に文字を書いて，そのデータをエンドポイントに引き渡して判定してみましょう．文字を実際に書いたら，下の `predict` メソッドを実行します．

In [ ]:
from IPython.display import HTML
HTML(open("input.html").read())

In [ ]:
print(data)

In [ ]:
response = predictor.predict(data)
print int(response)

## エンドポイントの削除

全て終わったら，エンドポイントを削除します．

In [ ]:
sagemaker.Session().delete_endpoint(predictor.endpoint)